# Od podatkov do podatkovnih tabel

Podatki za analizo lahko zbiramo na različne načine. Lahko jih pripravimo iz lastnih meritev in jih sami vnašamo v `pandas` v obliki zaporedij in podatkovnih tabel. Lahko jih pridobimo v obliki preglednic Excel iz nekega standardnega vira statističnih podatkov, kot je [podatkovna baza SiStat](https://pxweb.stat.si/) Statističnega urada Republike Slovenije, [podatkovna baza Eurostat](https://ec.europa.eu/eurostat/data/database) evropskega statističnega urada ali pa [podatkovna baza undata](https://data.un.org/) Organizacije združenih narodov. Spletna enciklopedija [Wikipedia](https://www.wikipedia.org/) je tudi bogat vir podatkovnih tabel, kot je na primer [tabela s podatki o bruto domačem proizvodu držav](https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)).

Kot bomo spoznali v nadaljevanju, `pandas` ponuja priročne funkcije za branje podatkov iz datotek s preglednicami Excel ali iz podatkovnih tabel v spletni enciklopediji Wikipedia. Pridobivanje takih podatkov in njihovo branje oziroma transformacija v `pandas` podatkovne tabele je zato relativno enostavno. Ker se običajno zaplete je, da imajo podatki pridobljeni iz različnih podatkovnih virov različno _strukturo_.

## Uvoz neurejenih podatkov

Profesorica, ki na FMF izvaja dve-semestrski predmet, si je v datoteki [`rezultati-kolokvijev.xlsx`](https://kt.ijs.si/~ljupco/lectures/papvp-2324/rezultati-kolokvijev.xlsx) zapisovala rezultate kolokvijev v prvem in drugem semestru. Naš cilj je napisati program v `pandas`, ki prebere podatke iz vseh listov preglednice in jih uredi v podatkovne tabele za analizo.

Uvoz podatkov iz preglednic v programu Excel nam omogoča funkcija [`read_excel`](https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html). Običajno uporabimo prva dva argumenta funkcije, in sicer:

  * `io` določa enotni naslov (URL) podatkovnega vira, t.j., spletni naslov ali pa pot na disku do preglednice Excel, ki jo želimo uvoziti. Argument nima privzete vrednosti.

  * `sheet_name` določa pozicijski indeks (celoštevilska vrednost) ali ime lista (niz znakov) iz preglednice, ki ga hočemo uvoziti. Lahko določimo tudi seznam, ki podaja kombinacijo imen in pozicijskih indeksov želenih listov. Z vrednostjo argumenta `None` zahtevamo uvoz vseh listov v slovar podatkovnih tabel. Privzeta vrednost argumenta je `0`, kar pomeni, da uvozimo prvi list preglednice.

V našem primeru rabimo vse liste, zato bomo preglednico uvozili takole:

In [4]:
import pandas as pd

vse_pt = pd.read_excel(
    "https://kt.ijs.si/~ljupco/lectures/papvp-2324/rezultati-kolokvijev.xlsx",
    sheet_name = None
)
print(vse_pt)

{'prvi kolokvij':       ime  telefon spol in starost  prvi semester  drugi semester
0     Ana      431            ž-19             80              97
1  Branko      720            m-20             78              74
2   David      141            m-19             75              68
3     Eva      210            ž-20             63              82
4   Franc      592            m-21             95              82, 'drugi kolokvij':       ime  telefon spol in starost  prvi semester  drugi semester
0     Ana      431            ž-19             82              95
1  Cvetka      761            ž-21             63              87
2   David      141            m-19             92              81
3     Eva      210            ž-20             82              63
4   Franc      592            m-21             99              54}


Rezultat je torej slovar dveh elementov, kjer sta ključa imena listov (`prvi kolokvij` in `drugi kolokvij`), vsaka vrednost je podatkovna tabela s podatki iz ustreznega lista v preglednici.

Če bi podatke hoteli uvoziti iz datoteke tipa CSV (angl. _comma separated values_, z vejico ločene vrednosti), bi lahko uporabili funkcijo [`read_csv`](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html), ki je še bolj enostavna za uporabo kot zgoraj opisana funkcija za uvoz podatkov iz preglednice Excel.

Če bi obe vrednosti v zgornjem slovarju takoj združili v skupno tabelo, bi zgubili informacijo o zaporedni številki kolokvija, na katerega se nanašajo podatki. Funkcija, ki nam omogoča združevanje tabel po vrsticah je `concat`:

In [5]:
pt = pd.concat(vse_pt.values())
print(pt)

      ime  telefon spol in starost  prvi semester  drugi semester
0     Ana      431            ž-19             80              97
1  Branko      720            m-20             78              74
2   David      141            m-19             75              68
3     Eva      210            ž-20             63              82
4   Franc      592            m-21             95              82
0     Ana      431            ž-19             82              95
1  Cvetka      761            ž-21             63              87
2   David      141            m-19             92              81
3     Eva      210            ž-20             82              63
4   Franc      592            m-21             99              54


Zato se bomo združevanja lotili drugače, najprej bomo vsaki podatkovni tabeli v slovarju dodali nov stolpec `kolokvij`, ki si zapomni zaporedno številko kolokvija (pravzaprav ime lista v preglednici), nato pa uporabili `concat` za združevanje:

In [8]:
for list in vse_pt:
    vse_pt[list]["kolokvij"] = list
pt = pd.concat(vse_pt.values())
print(pt)

      ime  telefon spol in starost  prvi semester  drugi semester  \
0     Ana      431            ž-19             80              97   
1  Branko      720            m-20             78              74   
2   David      141            m-19             75              68   
3     Eva      210            ž-20             63              82   
4   Franc      592            m-21             95              82   
0     Ana      431            ž-19             82              95   
1  Cvetka      761            ž-21             63              87   
2   David      141            m-19             92              81   
3     Eva      210            ž-20             82              63   
4   Franc      592            m-21             99              54   

         kolokvij  
0   prvi kolokvij  
1   prvi kolokvij  
2   prvi kolokvij  
3   prvi kolokvij  
4   prvi kolokvij  
0  drugi kolokvij  
1  drugi kolokvij  
2  drugi kolokvij  
3  drugi kolokvij  
4  drugi kolokvij  


Na tem mestu se zdi, da smo podatke že uvozili in da lahko začnemo z njihovo analizo. A temu ni tako, saj moramo podatke urediti. Namreč, zgornja tabela ima nekaj zelo očitnih pomanjkljivosti:

  * Vrednosti opazovanj za `spol` in `starost` so združene skupaj v vrednosti enega stolpca `spol in starost`.

  * Osnovni podatki o študentih se po nepotrebnem ponovijo dvakrat.

  * Podatek o zaporedni številki kolokvija je podan kot vrednost stolpca, podatek o zaporedni številki semestra pa kot ime stolpca.

Preden nadaljujemo z odpravljanjem teh težav, spoznajmo najprej pojem _urejenih podatkov_.

## Urejeni podatki (angl. _tidy data_)

Dogovor o urejenih podatkih določa **tri enostavna pravila za podatkovne tabele**.

  1. Vsak tip opazovane enote tvori svojo podatkovno tabelo.

  1. Vsaka opazovana enota tvori vrstico.

  1. Vsaka lastnost opazovanih enot je spremenljivka, ki tvori stolpec.

Pri načrtovanju naših podatkovnih tabel začnemo s premislekom o prvem pravilu. Ključno vprašanje pri tem premisleku je **Kateri tipi opazovanih enot se pojavljajo v naših podatkih?** V našem primeru opažamo, da en del podatkov (ki se tudi po nepotrebnem podvaja) podaja osnovne podatke o študentih. Drugi del podatkov pa podaja rezultate, ki so jih dosegli na kolokvijih. Zato bi podatke ločili v dva tipa opazovanih enot oziroma v dve podatkovni tabeli:

  * `studenti` z osnovnimi podatki o študentih, ki niso vezani na njihove dosežke na kolokvijih; in
  
  * `studenti_ocene` s podatki o dosežkih študentov na kolokvijih.
  
Drugo in tretje pravilo nama pomagata pri premisleku o vsebini posameznih tabel. Vsaka vrstica prve tabele `studenti` bo očitno vsebovala osnovne podatke o študentih. Vsaka vrstica v tej tabeli bo ustrezala enemu študentu, imela bo torej šest vrstic. Naši podatki iz preglednice vsebujejo štiri osnovne podatke o vsakem študentu: ime, telefon, spol in starost. Slednji seznam torej določa naše opazovane spremenljivke, zato bo tabela imela štiri ustrezno poimenovane stolpce. Torej prvi dve vrstici tabele `studenti` bosta:

| ime | telefon | spol | starost |
|:--|--:|:--:|--:|
| Ana | 431 | ž | 19 |   
| Branko | 720 | m | 20 |

Spremenljivke v zgornji tabeli lahko razdelimo v dve skupini. V prvi skupini je spremenljivka `ime`, ki je **fiksna spremenljivka**, katere vrednost določa predmet opazovanja, v našem primeru tip objekta študent. Imenujemo jo tudi **dimenzijska spremenljivka**. Druge spremenljivke sodijo v skupino **merjenih spremenljivk**: njihove vrednost določimo z opazovanjem ali merjenjem izbranega predmeta opazovanja, v našem primeru z opazovanjem (ugotavljanjem) različnih podatkov o študentih. Imenujemo jih tudi **vrednostne spremenljivke**.

Pri strukturi druge tabele `studenti_ocene` je potreben malo daljši premislek, ker je možnosti več. Ena, enostavna možnost je, da bi vsaka vrstica v tabeli bila en študent. Za vsakega študenta bi opazovali štiri spremenljivke, ki ustrezajo posameznim kolokvijem, dvema kolokvijema v prvem in dvema v drugem semestru. Taki predstavitvi podatkov se izogibamo in raje sledimo standardom (ali bolj dogovorom) za urejanje podatkov za analizo.

En tak standard imenujemo *urejeni podatki* (angl. _tidy data_): ni ravno standard, temveč dogovor o standardni obliki zapisovanja podatkov v podatkovne tabele, ki omogoča njihovo enostavno analizo s pomočjo standardnih funkcij za podatkovno analizo. Razvit je bil sicer za funkcije iz R-jevskih paketov knjižnice [`tidyverse`](https://www.tidyverse.org/). Lahko ga uporabljamo tudi za podatkovno analizo s funkcijami Pythonovske knjižnice `pandas`, ki posnemajo funkcionalnost paketov iz `tidyverse` v R-ju. Več o standardu _urejeni podatki_ lahko preberete v članku Wigham, H. (2014) Tidy Data. _Jorunal of Statistical Software 59(10)_: 1-23. DOI: [10.18637/jss.v059.i10](https://doi.org/10.18637/jss.v059.i10). Koristno branje je tudi članek [Data set](https://en.wikipedia.org/wiki/Data_set) v spletni enciklopediji Wikipedia.

Predlog zgoraj, da bi imeli za vsakega študenta štiri stolpce ne ustreza dogovoru _tidy data_. Slednji namreč določa, da spremenljivke ne ustrezajo opazovanim vrednostim v podatkih. Po taki interpretaciji stolpec `prvi kolokvij` ne ustreza pravi spremenljivki v urejenih podatkih, saj `prvi` je opazovana vrednost spremenljivke `semester`. Če upoštevamo to pravilo, potem bo imela podatkovna tabela z dosežki študentov štiri spremenljivke in ustrezne stolpce `ime`, `semester`, `kolokvij` in `rezultat`. Poglejmo prvih nekaj vrstic take tabele:

| ime | semester | kolokvij | rezultat |
|:--|--:|--:|--:|
| Ana | 1 | 1 | 80 |
| Ana | 2 | 1 | 97 |
| Ana | 1 | 2 | 82 |
| Ana | 2 | 2 | 95 |
| Branko | 1 | 1 | 78 |
| Branko | 2 | 1 | 74 |

V tem primeru imam opravka s tremi dimenzijskimi spremenljivkami, ki določajo konkreten kolokvij (ime študentke, semester, zaporedna številka kolokvija v semestru) in eno vrednostno spremenljivko - rezultat, ki ga je študent dosegel na kolokviju določenem z vrednostmi dimenzijskih spremenljivk.

Na tem mestu lahko tudi bolj formalno, matematično opredelimo osnovne pojme v sestavljeni tabeli:

  * **Podatkovna tabela** zapisuje podatke o opazovanih objektih $o$. Množico opazovanih objektov označimo z $O$. Za vsak opazovani objekt torej velja $o \in O$.

  * **Spremenljivka** $X$ je preslikava $X\colon O \to D_X$, kjer je $D_X$ domena spremenljivke, t.j., množica vseh njenih možnih vrednosti. Vrednost spremenljivke $\textrm{v}_X(o)$ za podan objekt $o$ dobimo torej kot rezultat preslikave (sliko objekta $o$), $X\colon o \mapsto \textrm{v}_X(o)$, pri čemer velja $\textrm{v}_X(o) \in D_X$.

  * Podatkovna tabela vsebuje $p$ spremenljivk: $d$ dimenzijskih in $p-d$ vrednostnih. Dimenzijske spremenljivke običajno ustrezajo prvim $d$ stolpcem podatkovne tabele $X_1, X_2, \dots, X_d$, zadnjih $p-d$ stolpcev pa ustreza vrednostnim spremenljivkam $X_{d+1}, \dots, X_{p}$.

  * Podatkovna tabela je tabelarična predstavitev preslikave $f\colon D_1 \times D_2 \times \dots \times D_d \to D_{d+1} \times \dots \times D_{p} $, pri čemer je vsak $D_i$ domena ustrezne spremenljivke $X_i$.

  * Vsaka vrstica podatkovne tabele podaja vrednosti $\textrm{v}_{X_i}(o)$ vseh spremenljivk $X_1, X_2, \dots X_p$ za opazovani objekt $o$.